In [2]:
with open('sequences20') as handle:
    u = handle.read()
import json
seqs_by_length = json.loads(u)

In [3]:
import numpy as np

In [4]:
# The matrix for an EVEN LENGTH R-L sequence
def cf2matrix(a):
    u = np.eye(2, dtype=np.int64)
    for x in a:
        u = np.matmul(u, [[x, 1], [1, 0]])
    return u

# The matrix for an even or odd length sequence, where the odd case is just the "condensed form" for an even sequence
def m_rep(seq):
    if len(seq) % 2 == 0:
        return cf2matrix(seq)
    else:
        return cf2matrix(seq + seq)
    
def rotl(x, k):
    return x[k:] + x[:k]

def min_rot(x):
    return min(rotl(x, k) for k in range(len(x)))

def min_rot2(x):
    return min(rotl(x, k) for k in range(0, len(x), 2))
    
def trace2length(t):
    return 2 * np.arccosh(t/2)

def length2trace(l):
    return 2 * np.cosh(l/2)

# The sequences that we actually want, from the "condensed forms" that take primitive roots in PGL_2(Z)
def actual_sequences(a, length):
    min_trace = int(length2trace(length)) + 1
    max_trace = int(length2trace(length+1)) + 1
    result = []
    for _x in a:
        x = _x[:] # Need this to make x harmlessly mutable
        m = m_rep(x)
        t = np.trace(m)
        if t < min_trace or t >= max_trace:
            x_length = trace2length(t)
            k = int((length + 1)/x_length)
            x *= k
        # Symmetrize by both R-L flip and time reversal, and add all four
        if len(x) % 2 == 0:
            result += [x, rotl(x, 1)]
        # Expand condensed form, and then symmetrize by time reversal
        else:
            xx = x + x
            result += [xx]
    return result

def my_actual_sequences(length):
    return actual_sequences(seqs_by_length[length], length)

def reduce_and_remove_duplicates(seqs):
    u = {tuple(min_rot2(x)) for x in seqs}
    return [list(x) for x in u]
            
def guys_for_trace(t):
    l = int(trace2length(t))
    l_seqs = my_actual_sequences(l)
    t_seqs = [x for x in l_seqs if np.trace(cf2matrix(x)) == t]
    return reduce_and_remove_duplicates(t_seqs)
    
def geos_in_trace_range(t0,t1):
    all_seqs = []
    for i in range(t0,t1+1):
        all_seqs += guys_for_trace(i)
    return all_seqs

In [11]:
geos_in_trace_range(10,20)

[[8, 1],
 [1, 1, 2, 1],
 [1, 8],
 [1, 1, 1, 2],
 [4, 2],
 [2, 4],
 [1, 9],
 [3, 3],
 [9, 1],
 [1, 10],
 [2, 5],
 [5, 2],
 [10, 1],
 [1, 11],
 [1, 1, 3, 1],
 [11, 1],
 [1, 1, 1, 3],
 [1, 2, 1, 2],
 [2, 1, 2, 1],
 [12, 1],
 [2, 6],
 [1, 12],
 [6, 2],
 [4, 3],
 [3, 4],
 [1, 1, 2, 2],
 [1, 13],
 [1, 2, 2, 1],
 [13, 1],
 [2, 7],
 [1, 1, 1, 4],
 [1, 14],
 [1, 1, 4, 1],
 [14, 1],
 [7, 2],
 [1, 15],
 [15, 1],
 [5, 3],
 [3, 5],
 [8, 2],
 [2, 8],
 [1, 16],
 [4, 4],
 [16, 1],
 [2, 1, 3, 1],
 [1, 2, 1, 3],
 [1, 1, 1, 1, 1, 1],
 [17, 1],
 [1, 1, 1, 5],
 [1, 17],
 [1, 1, 5, 1],
 [1, 2, 3, 1],
 [1, 3, 2, 1],
 [2, 9],
 [9, 2],
 [1, 18],
 [6, 3],
 [3, 6],
 [1, 1, 2, 3],
 [1, 1, 3, 2],
 [18, 1]]

In [12]:
inputList = [[1, 1, 2, 1], [1, 1, 1, 2], [8, 1], [1, 8], [4, 2], [2, 4], [9, 1], [1, 9], [10, 1], [5, 2], [2, 5], [1, 10], [1, 1, 3, 1], [1, 1, 1, 3], [11, 1], [1, 11], [12, 1], [4, 3], [3, 4], [1, 12], [6, 2], [2, 6], [13, 1], [1, 13], [14, 1], [7, 2], [2, 7], [1, 14], [1, 1, 4, 1], [1, 1, 1, 4], [15, 1], [5, 3], [3, 5], [1, 15], [2, 1, 3, 1], [1, 2, 1, 3], [16, 1], [1, 16], [8, 2], [2, 8], [1, 1, 5, 1], [1, 1, 1, 5], [17, 1], [1, 17], [1, 2, 3, 1], [1, 3, 2, 1], [1, 1, 2, 3], [1, 1, 3, 2], [18, 1], [9, 2], [2, 9], [1, 18], [6, 3], [3, 6]]

In [1]:
for seq in geos_in_trace_range(10,20):
    print(seq,seq in inputList(10,20))
        

NameError: name 'geos_in_trace_range' is not defined